# Politics in UNC

In [1]:
SCHOOL="unc"
SUBJECT="opinion"
START_YEAR=2010
FINAL_YEAR=2022
DATA_DIR="data"      # should be 'data'
OUTPUT_DIR="output"  # should be 'output'

In [2]:
%cd ..

/home/alexandersuen/diversity-colleges


In [3]:
%cd src

/home/alexandersuen/diversity-colleges/src


In [4]:
import sys
sys.path.append('src')
import ouraws
import nltk
import ourgraphs
import textutil

In [5]:
%cd ..

/home/alexandersuen/diversity-colleges


In [6]:
# NOTE: before loading, we need to be in the "diversity-colleges" folder

S3OBJECT_KEY = f"{DATA_DIR}/{SCHOOL}-{SUBJECT}-SNAPSHOT.parquet"
df = ouraws.getFromS3(S3OBJECT_KEY)

In [7]:
df.shape

(5791, 6)

In [8]:
df.head(2)

,title,url,body,year,month,day
0,Column: Practicing selfishness isn't inherentl...,https://www.dailytarheel.com/article/2023/03/o...,Column: Practicing selfishness isn't inherentl...,2023,3,1
1,Editorial: A tender(footed) apology from our E...,https://www.dailytarheel.com/article/2023/03/o...,Editorial: A tender(footed) apology from our E...,2023,3,1


In [9]:
import requests
import os

BIPARTISAN_API_KEY = os.environ.get("BIPARTISAN_API")
#BIPARTISAN_API_KEY = "gAAAAABeVpQJKRM5BqPX91XW2AKfz8pJosk182maAweJcm5ORAkkBFj__d2feG4H5KIeOKFyhUVSY_uGImiaSBCwy2L6nWxx4g=="
print(BIPARTISAN_API_KEY)
BIPARTISAN_URL = "https://api.thebipartisanpress.com/api/endpoints/beta/robert"

gAAAAABeVpQJKRM5BqPX91XW2AKfz8pJosk182maAweJcm5ORAkkBFj__d2feG4H5KIeOKFyhUVSY_uGImiaSBCwy2L6nWxx4g==


In [10]:
df2022 = df[df['year'] == 2022]

articles_list = df2022['body'].to_list()
for article in articles_list:
    payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
    response = requests.post(BIPARTISAN_URL, data=payload)
    print(response.text)
    break

-1.82093


In [11]:
df['year'].unique()

array([2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013,
       2012, 2011, 2010, 2009])

In [12]:
# SYNCHRONOUS METHOD : TAKES A LONG TIME

results = []

for year in range(2010, 2023):
    df2 = df[df['year'] == year]
    print(f"Year: {year} ... {df2.shape[0]}")

    articles_list = df2['body'].to_list()
    value_sum = 0.0
    article_count = 0
    for article in articles_list:
        payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
        response = requests.post(BIPARTISAN_URL, data=payload)
        try:
            value_sum += float(response.text)
            article_count += 1
            # print(f"{article_count}", end=".")
        except:
            pass  # this is a non-number
        
    results.append({
        'year' : year,
        'article_count' : article_count,
        'polarity_sum' : value_sum,
        'polarity_avg' : float(value_sum) / article_count
    })

    print(f"{year}\t{article_count}\t{value_sum}\t{value_sum/article_count}")
# for each year, get all articles' bodies & send to bipartisan press API 
#    and sum up the values & output the average for each year

Year: 2010 ... 898
2010	882	-160.55176099999983	-0.18203147505668915
Year: 2011 ... 971
2011	952	-477.56151900000026	-0.5016402510504204
Year: 2012 ... 835
2012	818	-498.4665029999999	-0.6093722530562345
Year: 2013 ... 477
2013	469	-672.7671650000002	-1.4344715671641795
Year: 2014 ... 234
2014	229	-103.76822700000001	-0.4531363624454149
Year: 2015 ... 225
2015	223	-223.8945550000001	-1.0040114573991037
Year: 2016 ... 330
2016	327	-1132.6694979999986	-3.463821094801219
Year: 2017 ... 331
2017	328	-1199.0271340000008	-3.6555705304878074
Year: 2018 ... 209
2018	209	-128.845006	-0.6164832822966507
Year: 2019 ... 252
2019	252	-299.306451	-1.1877240119047618
Year: 2020 ... 262
2020	262	-477.07351200000033	-1.8208912671755737
Year: 2021 ... 378
2021	378	-358.86650800000007	-0.9493822962962964
Year: 2022 ... 322
2022	322	-389.72263200000003	-1.2103187329192548


In [13]:
import pandas as pd

RESULTS_FILE = f"{DATA_DIR}/{SCHOOL}-POLARITY.parquet"

results_df = pd.DataFrame.from_records(results)
results_df.to_parquet(RESULTS_FILE)

In [ ]:
# ASYNCHRONOUS METHOD : should be a lot faster
import asyncio

results = []

for year in range(2010, 2023):
    df2 = df[df['year'] == year]
    print(f"Year: {year} ... {df2.shape[0]}")

    articles_list = df2['body'].to_list()
    value_sum = 0.0
    article_count = 0
    for article in articles_list:
        payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
        response = requests.post(BIPARTISAN_URL, data=payload)
        try:
            value_sum += float(response.text)
            article_count += 1
            # print(f"{article_count}", end=".")
        except:
            pass  # this is a non-number
        
    results.append({
        'year' : year,
        'article_count' : article_count,
        'polarity_sum' : value_sum,
        'polarity_avg' : float(value_sum) / article_count
    })

    print(f"{year}\t{article_count}\t{value_sum}\t{value_sum/article_count}")
# for each year, get all articles' bodies & send to bipartisan press API 
#    and sum up the values & output the average for each year